In [ ]:
# !pip install --upgrade gradio langgraph boto3

In [1]:
access_key_id = ""
secret_key = ""

In [2]:
import gradio as gr
import os
import boto3
from langchain_aws import ChatBedrockConverse
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [3]:
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
    aws_access_key_id=access_key_id,
    aws_secret_access_key=secret_key
)

llm = ChatBedrockConverse(
    client=bedrock_client,
    model_id="amazon.nova-lite-v1:0"
)


In [4]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    system_prompt = (
        "You are a helpful assistant. "
        "Answer all questions to the best of your ability."
    )
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": response}


# Define the node and edge
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add simple in-memory checkpointer
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [5]:
response = app.invoke(
    {"messages": [HumanMessage(content="Translate to Urdu: I love programming.")]},
    config={"configurable": {"thread_id": "1"}},
)

In [6]:
response.get('messages')[-1]

AIMessage(content='میں پروگرامنگ سے محبت کرتا ہوں۔\n\n(Note: The translation uses the masculine form "میں پروگرامنگ سے محبت کرتا ہوں". If the speaker is female, the sentence would be "میں پروگرامنگ سے محبت کرتی ہوں".)', additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'e782d6fd-6933-4cac-88c9-4a68b02e986c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 23 Apr 2025 10:28:28 GMT', 'content-type': 'application/json', 'content-length': '460', 'connection': 'keep-alive', 'x-amzn-requestid': 'e782d6fd-6933-4cac-88c9-4a68b02e986c'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': 547}}, id='run-80df881a-30a6-467a-a7f1-e37a242a6213-0', usage_metadata={'input_tokens': 24, 'output_tokens': 76, 'total_tokens': 100})

In [7]:
response = app.invoke(
    {"messages": [HumanMessage(content="What is 2+2")]},
    config={"configurable": {"thread_id": "1"}},
)

In [8]:
response.get('messages')[-1]

AIMessage(content='2 + 2 equals 4. This is a basic arithmetic operation where you add two quantities of 2 together to get a total of 4.', additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '560cb57c-8ab7-4368-a31d-d5217244510e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 23 Apr 2025 10:28:29 GMT', 'content-type': 'application/json', 'content-length': '299', 'connection': 'keep-alive', 'x-amzn-requestid': '560cb57c-8ab7-4368-a31d-d5217244510e'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': 259}}, id='run-7b964f57-d227-4321-8b58-cc805fb5e495-0', usage_metadata={'input_tokens': 111, 'output_tokens': 32, 'total_tokens': 143})

In [9]:
response

{'messages': [HumanMessage(content='Translate to Urdu: I love programming.', additional_kwargs={}, response_metadata={}, id='73328681-b981-4760-9cd1-ed7090ce7b76'),
  AIMessage(content='میں پروگرامنگ سے محبت کرتا ہوں۔\n\n(Note: The translation uses the masculine form "میں پروگرامنگ سے محبت کرتا ہوں". If the speaker is female, the sentence would be "میں پروگرامنگ سے محبت کرتی ہوں".)', additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'e782d6fd-6933-4cac-88c9-4a68b02e986c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 23 Apr 2025 10:28:28 GMT', 'content-type': 'application/json', 'content-length': '460', 'connection': 'keep-alive', 'x-amzn-requestid': 'e782d6fd-6933-4cac-88c9-4a68b02e986c'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': 547}}, id='run-80df881a-30a6-467a-a7f1-e37a242a6213-0', usage_metadata={'input_tokens': 24, 'output_tokens': 76, 'total_tokens': 100}),
  HumanMessage(content='What is 2+2', additional_kwargs={}, 

In [10]:
import gradio as gr

def call_graph(text):
    response = app.invoke(
    {"messages": [HumanMessage(content=text)]},
    config={"configurable": {"thread_id": "1"}},
    )
    return response['messages'][-1].content
    
demo = gr.Interface(
    fn=call_graph,
    inputs=gr.Textbox(label="Please enter query here", lines=5, max_lines=10),
    outputs=gr.Textbox(label = 'Response', lines = 10, max_lines = 20)
)

demo.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [11]:
import gradio as gr

def call_graph(text, user_name):
    response = app.invoke(
    {"messages": [HumanMessage(content=text)]},
    config={"configurable": {"thread_id": "1"}},
    )
    return response['messages'][-1].content
    
demo = gr.Interface(
    fn=call_graph,
    inputs=gr.Textbox(label="Please enter query here", lines=5, max_lines=10),
    outputs=gr.Textbox(label = 'Response', lines = 10, max_lines = 20)
)

demo.launch(share = True)


C:\Users\saqlain.shah\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\utils.py:999: UserWarning: Expected 2 arguments for function <function call_graph at 0x00000221578D0680>, received 1.
  warnings.warn(
C:\Users\saqlain.shah\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\utils.py:1003: UserWarning: Expected at least 2 arguments for function <function call_graph at 0x00000221578D0680>, received 1.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://e6471e79ece0eeedba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
